## **How to run the notebook**

We pre-run the entire notebook and kept the results of the different models.

However, if you want to run it you need to know a few things:
- We stored our data on Google Drive so in order to run the code you need to upload the Folder with the images in your Google Drive under the following path: *drive/MyDrive/Colab Notebooks/UCMerced_LandUse*


- We divided the code in 'Chapters' and we tried to make every single one indipendent from the rest of the code in order to be able to run them indipendently. In many of them we put a sub-chapter to load the right data to run the model.

# **Load the Augmented Data with fill_mode = 'constant'**

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
import numpy as np

 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'drive/MyDrive/Colab Notebooks/UCMerced_LandUse')

# img

# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

num_classes=21

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  

# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

# Test
test_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 1470 images belonging to 21 classes.
Found 315 images belonging to 21 classes.
Found 315 images belonging to 21 classes.


In [ ]:
# Create Dataset objects
# ----------------------

# Training
train_dataset_augm = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
train_dataset_augm = train_dataset_augm.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))


##CNN Model

**Comments**

The model is trained with Batches of size 32, and is composed by the following layers:

- Conv2D with 8 filters and a kernel of 3,3 
- ReLU
- MaxPool2D with pool size 2,2 

- Conv2D with 16 filters and a kernel of 3,3 
- ReLU
- MaxPool2D with pool size 2,2

- Conv2D with 32 filters and a kernel of 3,3
- ReLU
- MaxPool2D with pool size 2,2

- Flatten
- Dense of 300 units with Sigmoid function
- Dense of 21 units with Softmax function

In [ ]:
model_cnn = tf.keras.Sequential()

model_cnn.add(tf.keras.layers.Conv2D(input_shape=(256, 256, 3),
                                 filters=8,
                                 kernel_size=(3, 3),        
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU()) 
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model_cnn.add(tf.keras.layers.Conv2D(filters=16,
                                 kernel_size=(3, 3),
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model_cnn.add(tf.keras.layers.Conv2D(filters=32,
                                 kernel_size=(3, 3),
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model_cnn.add(tf.keras.layers.Flatten())
model_cnn.add(tf.keras.layers.Dense(units=300, activation=tf.keras.activations.sigmoid))
model_cnn.add(tf.keras.layers.Dense(units=21, activation=tf.keras.activations.softmax))

model_cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 256, 256, 8)       224       
                                                                 
 re_lu_12 (ReLU)             (None, 256, 256, 8)       0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 128, 128, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 re_lu_13 (ReLU)             (None, 128, 128, 16)      0         
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                 

In [ ]:
# Optimization params

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

In [ ]:
# Compile Model
model_cnn.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
import os
from datetime import datetime

# New callbacks (for names)
exps_dir = os.path.join(dataset_dir, 'Mark1_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = 'CNN'

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []


ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'checkpoint'), 
                                                  save_weights_only=True,  
                                                  monitor='val_accuracy',
                                                  mode='max',
                                                  save_best_only=False)
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

In [ ]:
model_cnn.fit(x=train_dataset_augm,
          epochs=100,  
          steps_per_epoch=int(np.ceil(train_gen.samples / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(valid_gen.samples / bs)), 
          callbacks=callbacks)

Epoch 1/100
46/46 [==============================] - 256s 6s/step - loss: 3.0491 - accuracy: 0.1034 - val_loss: 2.7022 - val_accuracy: 0.1714
Epoch 2/100
46/46 [==============================] - 27s 587ms/step - loss: 2.6579 - accuracy: 0.1905 - val_loss: 2.5132 - val_accuracy: 0.2349
Epoch 3/100
46/46 [==============================] - 27s 595ms/step - loss: 2.3604 - accuracy: 0.2850 - val_loss: 2.1736 - val_accuracy: 0.2984
Epoch 4/100
46/46 [==============================] - 27s 594ms/step - loss: 2.1214 - accuracy: 0.3333 - val_loss: 2.2744 - val_accuracy: 0.3333
Epoch 5/100
46/46 [==============================] - 27s 579ms/step - loss: 1.9871 - accuracy: 0.3857 - val_loss: 1.8460 - val_accuracy: 0.4190
Epoch 6/100
46/46 [==============================] - 27s 584ms/step - loss: 1.7789 - accuracy: 0.4476 - val_loss: 1.8289 - val_accuracy: 0.3778
Epoch 7/100
46/46 [==============================] - 27s 589ms/step - loss: 1.6436 - accuracy: 0.4810 - val_loss: 1.5989 - val_accuracy: 0

We get as resulta the following:

*loss: 0.0913 - accuracy: 0.9796 - val_loss: 0.5918 - val_accuracy: 0.8159*


# **Transfer Learning** 

- **InceptionV3:** *loss: 0.5634 - accuracy: 0.9687 - val_loss: 1.8383 - val_accuracy: 0.9302*

## InceptionV3

InceptionV3 was created by Google and it was called GoogLeNet.
At only 7 million parameters, it was much smaller than the then prevalent models like VGG and AlexNet

Augmented Data

**Results:** *loss: 0.5634 - accuracy: 0.9687 - val_loss: 1.8383 - val_accuracy: 0.9302*

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, models

InceptionV3_model = InceptionV3(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in InceptionV3_model.layers:
  layer.trainable = False

In [ ]:
# add new classifier layers
flatten_layer = layers.Flatten()
prediction_layer = layers.Dense(21, activation='softmax')

# define new model
InceptionV3_model_prediction = models.Sequential([
              InceptionV3_model,
              flatten_layer,
              prediction_layer
])

# summarize
InceptionV3_model_prediction.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 73728)             0         
                                                                 
 dense_1 (Dense)             (None, 21)                1548309   
                                                                 
Total params: 23,351,093
Trainable params: 1,548,309
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
InceptionV3_model_prediction.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10,  restore_best_weights=True)

In [ ]:
InceptionV3_model_prediction.fit(x=train_dataset_augm,
          epochs=100,
          steps_per_epoch=int(np.ceil(train_gen.samples / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(valid_gen.samples / bs)), 
          callbacks=[es])

Epoch 1/100
46/46 [==============================] - 722s 16s/step - loss: 5.7869 - accuracy: 0.6361 - val_loss: 2.3675 - val_accuracy: 0.7778
Epoch 2/100
46/46 [==============================] - 26s 566ms/step - loss: 1.5553 - accuracy: 0.8340 - val_loss: 1.5356 - val_accuracy: 0.8413
Epoch 3/100
46/46 [==============================] - 24s 522ms/step - loss: 1.1998 - accuracy: 0.8782 - val_loss: 1.9095 - val_accuracy: 0.8286
Epoch 4/100
46/46 [==============================] - 24s 521ms/step - loss: 1.2314 - accuracy: 0.8762 - val_loss: 2.1955 - val_accuracy: 0.8063
Epoch 5/100
46/46 [==============================] - 24s 525ms/step - loss: 1.2919 - accuracy: 0.8871 - val_loss: 1.9725 - val_accuracy: 0.8698
Epoch 6/100
46/46 [==============================] - 24s 522ms/step - loss: 1.8199 - accuracy: 0.8762 - val_loss: 1.7725 - val_accuracy: 0.8889
Epoch 7/100
46/46 [==============================] - 24s 525ms/step - loss: 1.3683 - accuracy: 0.8932 - val_loss: 1.2956 - val_accuracy: 